In [4]:
import numpy as np
from numba import jit, njit, vectorize, cfunc
from timeit import default_timer as timer

In [52]:
name_=['example4','example5','bin','rome']
name_n=3
Re = np.genfromtxt(name_[name_n]+'\\Re.csv', delimiter=',')
epsD = np.genfromtxt(name_[name_n]+'\\epsD.csv', delimiter=',')
Re

array([120027.16267111,   3729.48906014, 124198.45916958, ...,
         1225.35103377,    985.52416782,    354.43513416])

In [57]:
Np=10000000
Re=100*np.power(1e8/100,np.random.rand(Np))
epsD=1e-6*np.power(0.05/1e-6,np.random.rand(Np))

In [58]:
Np=len(Re)

Re_lam=np.where(Re<=2000,1,0)
Re_turb=np.where(Re>=4000,1,0)
Re_trans=np.ones(Np)-Re_lam-Re_turb

print('fraction of laminar pipes',np.sum(Re_lam)/Np)
print('fraction of transient pipes',np.sum(Re_trans)/Np)
print('fraction of turbulent pipes',np.sum(Re_turb)/Np)

fraction of laminar pipes 0.2167884
fraction of transient pipes 0.0502206
fraction of turbulent pipes 0.732991


In [63]:
#####################
# friction functions#
#####################

@njit
def f_swamee(epsD,Re):
    y1=5.74/np.power(Re,0.9)
    y2=epsD/3.7+y1
    c1=-2/np.log(10)
    y3=c1*np.log(y2)
    f=1/(y3*y3)

    y3=c1*np.log(y2)
    fp=(2*0.9*c1)*y1*f/(y2*y3*Re)
    
    return [f,fp]

@njit
def f_dunlop(epsD,Re):
    aa=-1.5634601348517065795
    #aa=-2*0.9*2/np.log(10)
    ab=0.00328895476345399058690
    #ab=5.74/np.power(4000,0.9)
    c2=-2/np.log(10)
    #c2=-0.8685889638065035
    
    ac=aa*ab
    
    y2=epsD/3.7+ab
    y3=c2*np.log(y2)
    
    fa=1/(y3*y3)
    fb=(2+ac/(y2*y3))*fa
    
    x1=7*fa-fb
    x2=0.128-17*fa+2.5*fb
    x3=-0.128+13*fa-(fb+fb)
    x4=0.032-3*fa+0.5*fb
    
    R=Re/2000
      
    f=x1+R*(x2+R*(x3+R*x4))
    fp=x2+R*(2*x3+3*R*x4)
    fp=fp/2000
    
    return [f,fp]

@njit
def f_lam(epsD,Re):
    return [64/Re ,-64/(Re*Re)]

@njit(parallel=True)
def f_tot(epsD_vec,Re_vec):
    N=len(Re_vec)
    f_vec=np.zeros(N)
    fp_vec=np.zeros(N)
    for i in prange(N):
        Re=Re_vec[i]
        epsD=epsD_vec[i]
        if Re<=2000:
            #############
            #laminar flow
            f=64/Re 
            fp=-64/(Re*Re)
        elif Re>=4000:
            #################
            ###turbulent flow
            #swamee jain#####
            y1=5.74/np.power(Re,0.9)
            y2=epsD/3.7+y1
            c1=-2/np.log(10)
            y3=c1*np.log(y2)
            f=1/(y3*y3)

            y3=c1*np.log(y2)
            fp=(2*0.9*c1)*y1*f/(y2*y3*Re)
        else:
            ######################
            # transition flow####
            #dunlop interpolation
            ######################
            aa=-1.5634601348517065795
            #aa=-2*0.9*2/np.log(10)
            ab=0.00328895476345399058690
            #ab=5.74/np.power(4000,0.9)
            c2=-2/np.log(10)
            #c2=-0.8685889638065035

            ac=aa*ab

            y2=epsD/3.7+ab
            y3=c2*np.log(y2)

            fa=1/(y3*y3)
            fb=(2+ac/(y2*y3))*fa

            x1=7*fa-fb
            x2=0.128-17*fa+2.5*fb
            x3=-0.128+13*fa-(fb+fb)
            x4=0.032-3*fa+0.5*fb

            R=Re/2000

            f=x1+R*(x2+R*(x3+R*x4))
            fp=x2+R*(2*x3+3*R*x4)
            fp=fp/2000
        f_vec[i]=f
        fp_vec[i]=fp
    return [f_vec,fp_vec]

@njit(parallel=True)
def f_tot1(epsD_vec,Re_vec,f1,f2,f3):
    N=len(Re_vec)
    f_vec=np.zeros(N)
    fp_vec=np.zeros(N)
    a=np.array([0,0])
    for i in prange(N):
        Re=Re_vec[i]
        epsD=epsD_vec[i]
        if Re<=2000:
            a=f1(epsD,Re)
            f=a[0]
            fp=a[1]
        elif Re>=4000:
            a=f3(epsD,Re)
            f=a[0]
            fp=a[1]            
        else:
            a=f2(epsD,Re)
            f=a[0]
            fp=a[1]            
        f_vec[i]=f
        fp_vec[i]=fp
    return [f_vec,fp_vec]
    #f_tot1(epsD,Re,f_lam,f_dunlop,f_swamee)

In [60]:
start = timer()
oo=np.ones(len(Re))

f=np.zeros(Np)
fp=np.zeros(Np)

Re_turb=np.zeros(Np)
Re_trans=np.zeros(Np)
Re_lam=np.zeros(Np)

a=np.zeros((Np,2))
a1=np.zeros((Np,2))
a2=np.zeros((Np,2))
a3=np.zeros((Np,2))

print('initialize variables, time=',timer() - start,'s')

initialize variables, time= 0.03655520000006618 s


In [61]:
start = timer()

Re_turb=np.maximum(Re,4000)
Re_trans=np.minimum(np.maximum(Re,2000),4000)
Re_lam=np.minimum(Re,2000)



a_turb=1*(Re>=4000)
a_trans=(Re<4000)*(Re>2000)*1
a_lam=1*(Re<=2000)




a1=f_swamee(epsD,Re)
a2=f_dunlop(epsD,Re)
a3=f_lam(epsD,Re)

a=a_turb*a1+a_trans*a2+a_lam*a3

a=a_turb*f_swamee(epsD,Re)+a_trans*f_dunlop(epsD,Re)+a_lam*f_lam(epsD,Re)


f=a[0]
fp=a[1]
print('Calculate f & fp, time=',timer() - start,'s')

%timeit a=np.where(Re>=4000,1,0)*f_swamee(epsD,Re)+np.where(abs(Re-3000)<1000,1,0)*f_dunlop(epsD,Re)+np.where(Re<=2000,1,0)*f_lam(epsD,Re)

Calculate f & fp, time= 7.280867100000023 s
2.45 s ± 48.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:

a=f_tot(epsD,Re)

start = timer()
a=f_tot(epsD,Re)
f=a[0]
fp=a[1]
print('Calculate f & fp, time=',timer() - start,'s')
%timeit f_tot(epsD,Re) #first iteration is slow because of warmup? 
%timeit f_tot1(epsD,Re,f_lam,f_dunlop,f_swamee)

Calculate f & fp, time= 0.1865023000000292 s
193 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.99 s ± 29.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
a=f_tot(epsD,Re)
b=f_tot1(epsD,Re,f_lam,f_dunlop,f_swamee)


array([0., 0., 0., ..., 0., 0., 0.])

In [48]:
Np=10000000
Re=100*np.power(1e8/100,np.random.rand(Np))
epsD=1e-6*np.power(0.05/1e-6,np.random.rand(Np))

In [49]:
%timeit f_tot(epsD,Re)
%timeit np.where(Re>=4000,1,0)*f_swamee(epsD,Re)+np.where(abs(Re-3000)<1000,1,0)*f_dunlop(epsD,Re)+np.where(Re<=2000,1,0)*f_lam(epsD,Re)

752 ms ± 8.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.06 s ± 39.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
start = timer()
a=f_tot(epsD,Re)
print(timer() - start)

start = timer()
a1=f_swamee(epsD,Re)
a2=f_dunlop(epsD,Re)
a3=f_lam(epsD,Re)
a10=np.where(Re>=4000,1,0)
a20=np.where(abs(Re-3000)<1000,1,0)
a30=np.where(Re<=2000,1,0)
b=a1*a10+a2*a20+a3*a30
print(timer() - start)



0.7280911000000287
1.0134945999999445


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

183 ms ± 2.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


567 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
422 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


array([0.0126214 , 0.01964285, 0.10846013, ..., 0.00648425, 0.02269704,
       0.01236301])

0.032

array([0.04515778, 0.04515778])

In [2]:
import numpy as np
from numba import jit, njit, vectorize, cfunc, prange
from timeit import default_timer as timer

Np=10000000
x_vec=100*np.power(1e8/100,np.random.rand(Np))

@njit
def f1(x):
    return 64/x

@njit
def f2(x):
    #x is the Reynolds number(Re), y is the Darcy friction(f)
    #for transition, we can assume Re=4000 (max possible friction)
    y=0.02
    y=(-2/np.log(10))*np.log(2.51/(4000*np.sqrt(y)))
    return 1/(y*y)

@njit
def f3(x): #colebrook-white approximation
    #x is the Reynolds number(Re), y is the Darcy friction(f)
    y=0.02
    y=(-2/np.log(10))*np.log(2.51/(x*np.sqrt(y)))
    return 1/(y*y)



%timeit f(x_vec,f1,f2,f3)
a=f(x_vec,f1,f2,f3)

@njit(parallel=True)
def f(x_vec):
    N=len(x_vec)
    y_vec=np.zeros(N)
    for i in prange(N):
        x=x_vec[i]
        if x<=2000:
            y=64/x
        elif x>=4000:
            y=0.02
            y=(-2/np.log(10))*np.log(2.51/(x*np.sqrt(y)))
            y=1/(y*y)
        else:
            y=0.02
            y=(-2/np.log(10))*np.log(2.51/(4000*np.sqrt(y)))
            y=1/(y*y)
        y_vec[i]=y
    return y_vec

%timeit f(x_vec)  #0.181 seconds 
b=f(x_vec)

a-b

start=timer()

y = np.empty_like(x_vec)

a1=np.where(x_vec<=2000,True,False)
a3=np.where(x_vec>=4000,True,False)
a2=~(a1 | a3)

y[a1] = f1(x_vec[a1])
y[a2] = f2(x_vec[a2])
y[a3] = f3(x_vec[a3])

print(timer() - start)

y-a

def ff(x): #pure python
    y = np.empty_like(x)
    
    mask = x <= 2000
    y[mask] = 64 / x[mask]
    
    mask = (x > 2000) & (x < 4000)
    y[mask] = np.log(1.2 * x[mask])
    
    mask = x >= 4000
    y[mask] = np.log(x[mask])

    return y

55.3 ms ± 4.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
52.1 ms ± 8.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.146432400000009


array([0., 0., 0., ..., 0., 0., 0.])

In [67]:
Np=10000000
x_vec=100*np.power(1e8/100,np.random.rand(Np))

@njit(parallel=True)
def f(x_vec):
    N=len(x_vec)
    y_vec=np.zeros(N)
    for i in prange(N):
        x=x_vec[i]
        if x<=2000:
            y=64/x
        elif x>=4000:
            y=0.02
            y=(-2/np.log(10))*np.log(2.51/(x*np.sqrt(y)))
            y=1/(y*y)
        else:
            y=0.02
            y=(-2/np.log(10))*np.log(2.51/(4000*np.sqrt(y)))
            y=1/(y*y)
        y_vec[i]=y
    return y_vec

%timeit f(x_vec)  #0.181 seconds 


70 ms ± 6.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
